<a href="https://colab.research.google.com/github/Samyuktha0104/Document-Processing/blob/main/Integarted_NLP_Project_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing and Installing libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import gensim.downloader as api

# Load the Word2Vec model from Gensim API and save it to Google Drive
v2w_model = api.load('word2vec-google-news-300')
v2w_model.save('/content/drive/My Drive/w2vecmodel.mod')  # Adjust the path as needed


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
!pip install pytesseract pillow python-docx pymupdf
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install neo4j
!pip install python-docx
!pip install sentencepiece
!pip install transformers
!pip install nltk # Install nltk if you haven't already

import nltk # Import nltk at the beginning
nltk.download('punkt') # Now you can download punkt
nltk.download('stopwords')

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import files
from PIL import Image
import pytesseract
import re
import numpy as np
from docx import Document
import fitz  # PyMuPDF
import os

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim import corpora, models
from difflib import SequenceMatcher



# Download nltk punkt tokenizer - This is redundant, you've already downloaded it above
# nltk.download('punkt')
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 7.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Knowledge Graph

In [5]:
# Required Imports
from PIL import Image
import pytesseract
from docx import Document
import fitz  # PyMuPDF
import os
import spacy
from google.colab import files
from neo4j import GraphDatabase
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import networkx as nx

# Neo4j connection details
uri = "neo4j+s://efac17af.databases.neo4j.io"
username = "neo4j"
password = "9S2uREJTWlq3iMZjSs4c0be1apXT0rc7oockbBrQEs4"

# Initialize Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# List of ambiguous terms to ignore
ambiguous_terms = {"it", "this", "that", "which", "they", "their", "its", "the"}

# Text Extraction Functions
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

# Preprocess file to extract text
def preprocess_file(file_path):
    extension = os.path.splitext(file_path)[1].lower()
    if extension in ['.jpg', '.jpeg', '.png']:
        text = extract_text_from_image(file_path)
    elif extension == '.txt':
        text = extract_text_from_txt(file_path)
    elif extension == '.docx':
        text = extract_text_from_docx(file_path)
    elif extension == '.pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")
    return text

# Entity and Relationship Extraction
def extract_entities_and_relations(text):
    doc = nlp(text)
    entities = [(ent.text.strip().lower(), ent.label_) for ent in doc.ents if ent.text.strip().lower() not in ambiguous_terms]
    entity_dict = {ent[0]: ent[1] for ent in entities}  # Maps names to labels for quick reference

    relations = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.head.pos_ == 'VERB':
            for child in token.head.children:
                if child.dep_ == 'dobj':  # direct object
                    subj = token.text.strip().lower()
                    relation = token.head.text.strip().lower()
                    obj = child.text.strip().lower()

                    if subj not in entity_dict:
                        entities.append((subj, "UNKNOWN"))
                        entity_dict[subj] = "UNKNOWN"
                    if obj not in entity_dict:
                        entities.append((obj, "UNKNOWN"))
                        entity_dict[obj] = "UNKNOWN"

                    relations.append((subj, relation, obj))
    return entities, relations

# Normalize and Deduplicate Entities
def normalize_and_deduplicate_entities(entities):
    unique_entities = set(entities)
    final_entities = list(unique_entities)
    return final_entities

# Add Entities to Neo4j
def add_entities_to_graph(entities):
    with driver.session() as session:
        for entity, label in entities:
            session.run("MERGE (e:Entity {name: $name, label: $label})", name=entity, label=label)

# Add Relationships to Neo4j (Only if Both Nodes Exist)
def add_relations_to_graph(relations):
    with driver.session() as session:
        for subj, relation, obj in relations:
            subj_exists = session.run("MATCH (a:Entity {name: $name}) RETURN a", name=subj).single()
            obj_exists = session.run("MATCH (b:Entity {name: $name}) RETURN b", name=obj).single()
            if subj_exists and obj_exists:
                session.run("""
                    MATCH (a:Entity {name: $subj})
                    MATCH (b:Entity {name: $obj})
                    MERGE (a)-[:RELATION {type: $relation}]->(b)
                """, subj=subj, obj=obj, relation=relation)

# Fetch Nodes and Relationships from Neo4j for Visualization
def fetch_graph_data():
    with driver.session() as session:
        nodes_query = "MATCH (n:Entity) RETURN n.name AS name, n.label AS label"
        nodes_data = session.run(nodes_query)
        relations_query = "MATCH (a:Entity)-[r:RELATION]->(b:Entity) RETURN a.name AS from, b.name AS to, type(r) AS relation"
        relations_data = session.run(relations_query)

        nodes = [(record["name"], {"label": record["label"]}) for record in nodes_data]
        edges = [(record["from"], record["to"], {"relation": record["relation"]}) for record in relations_data]
        return nodes, edges

# Build NetworkX Graph for Visualization
def build_networkx_graph(nodes, edges):
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    return G

# Visualize Graph using Matplotlib
def visualize_graph(G):
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G, k=0.5)
    nx.draw(G, pos, with_labels=True, node_size=3000, node_color="#89CFF0", font_size=10, font_weight="bold", arrows=True)
    edge_labels = nx.get_edge_attributes(G, 'relation')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
    plt.title("Knowledge Graph Visualization")
    plt.show()

# Main knowledge graph function
def knowledge_graph():
    clear_output()
    display(HTML('''
    <div style="background-color: #e8f5e9; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #388e3c;"><i class="fa fa-share-alt" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Executing Knowledge Graph Functionality...</h3>
        <p>Building a comprehensive knowledge graph based on extracted entities and relationships.</p>
    </div>
    '''))

    # Prompt user to upload a file
    print("Please upload a file:")
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    try:
        # Extract text from uploaded file
        extracted_text = preprocess_file(file_path)
        print("Extracted Text:")
        print(extracted_text)

        # Process text to extract entities and relationships
        entities, relations = extract_entities_and_relations(extracted_text)
        final_entities = normalize_and_deduplicate_entities(entities)

        # Display extracted entities
        display(HTML('<h4 style="color: #388e3c;">Extracted Entities:</h4>'))
        for entity in final_entities:
            display(HTML(f'<p style="font-family: Arial; color: #4caf50;">Entity: {entity[0]}, Label: {entity[1]}</p>'))

        # Add entities and relationships to Neo4j
        add_entities_to_graph(final_entities)

        # Display relationship addition
        display(HTML('<h4 style="color: #388e3c;">Adding Relationships:</h4>'))
        for subj, relation, obj in relations:
            display(HTML(f'<p style="font-family: Arial; color: #388e3c;">Attempting to add relationship: {subj} -[{relation}]-> {obj}</p>'))
            add_relations_to_graph([(subj, relation, obj)])

        # Fetch data from Neo4j and build the graph
        nodes, edges = fetch_graph_data()
        G = build_networkx_graph(nodes, edges)

        # Visualize the graph using Matplotlib
        visualize_graph(G)

    except ValueError as e:
        print(e)
    finally:
        driver.close()


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


# Key phrase extraction and Summarisation

In [6]:




def keyphrase_extraction_summarization():
    clear_output()
    display(HTML('''
    <div style="background-color: #fff3e0; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #f57c00;"><i class="fa fa-key" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Executing Keyphrase Extraction and Summarization...</h3>
        <p>Identifying key phrases and summarizing the main points from the text.</p>
    </div>
    '''))

    # Upload the file
    uploaded = files.upload()
    file_path = next(iter(uploaded))  # Get the uploaded file path

    # Extract text based on file type
    def preprocess_file(file_path):
        extension = os.path.splitext(file_path)[1].lower()
        if extension in ['.jpg', '.jpeg', '.png']:
            image = Image.open(file_path)
            return pytesseract.image_to_string(image)
        elif extension == '.txt':
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        elif extension == '.docx':
            doc = Document(file_path)
            return "\n".join([paragraph.text for paragraph in doc.paragraphs])
        elif extension == '.pdf':
            text = ""
            with fitz.open(file_path) as pdf:
                for page in pdf:
                    text += page.get_text("text")
            return text
        else:
            raise ValueError("Unsupported file format")

    # Extract text
    try:
        extracted_text = preprocess_file(file_path)
    except ValueError as e:
        display(HTML(f"<p style='color:red;'>{e}</p>"))
        return

    # Keyphrase extraction and summarization
    def preprocess_text(text):
        doc = nlp(text.lower())
        tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
        return " ".join(tokens)

    def extract_candidate_keyphrases(text):
        doc = nlp(text)
        candidates = []
        for chunk in doc.noun_chunks:
            if len(chunk.text.split()) <= 3:
                candidates.append(chunk.text)
        for token in doc:
            if token.pos_ in {"NOUN", "PROPN", "ADJ"} and len(token.text) > 1:
                candidates.append(token.text)
        return candidates

    def rank_keyphrases(text, num_phrases=10):
        processed_text = preprocess_text(text)
        candidates = extract_candidate_keyphrases(processed_text)
        vectorizer = TfidfVectorizer()
        vectorizer.fit_transform([processed_text])
        phrase_counts = Counter(candidates)
        ranked_phrases = sorted(phrase_counts.items(), key=lambda x: x[1], reverse=True)
        return [phrase for phrase, _ in ranked_phrases[:num_phrases]]

    def split_into_sentences(text):
        import nltk
        nltk.download('punkt')
        return nltk.sent_tokenize(text)

    def score_sentences(text, keyphrases, max_words=50):
        sentences = split_into_sentences(text)
        vectorizer = CountVectorizer(vocabulary=keyphrases)
        sentence_vectors = vectorizer.fit_transform(sentences)
        transformer = TfidfTransformer()
        tfidf_matrix = transformer.fit_transform(sentence_vectors)
        sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
        sorted_indices = sentence_scores.argsort()[::-1]
        summary_sentences = []
        word_count = 0
        for idx in sorted_indices:
            sentence = sentences[idx]
            sentence_word_count = len(sentence.split())
            if word_count + sentence_word_count <= max_words:
                summary_sentences.append(sentence)
                word_count += sentence_word_count
            else:
                break
        return " ".join(summary_sentences)

    # Run keyphrase extraction and summarization
    keyphrases = rank_keyphrases(extracted_text)
    summary = score_sentences(extracted_text, keyphrases, max_words=100)

    # Display results in a styled format
    display(HTML(f'''
    <div style="background-color: #ffebee; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #d32f2f;"><i class="fa fa-lightbulb-o" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Extracted Keyphrases</h3>
        <ul style="color: #555; font-size: 16px;">
            {"".join([f"<li>{phrase}</li>" for phrase in keyphrases])}
        </ul>
    </div>
    '''))

    display(HTML(f'''
    <div style="background-color: #e3f2fd; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #1976d2;"><i class="fa fa-file-text" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Generated Summary</h3>
        <p style="color: #555; font-size: 16px;">{summary}</p>
    </div>
    '''))

# Title and Abstract Generation

In [7]:
from google.colab import files
from IPython.display import HTML, display
import pytesseract
from PIL import Image
from docx import Document
import fitz  # PyMuPDF for PDFs
import spacy
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np

# Load SpaCy model and NLTK resources
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def extract_candidate_keyphrases(text):
    doc = nlp(text)
    candidates = [chunk.text for chunk in doc.noun_chunks]
    candidates.extend([token.text for token in doc if token.pos_ in {"NOUN", "PROPN", "ADJ"}])
    return candidates

def rank_keyphrases(text, num_phrases=5):
    processed_text = preprocess_text(text)
    candidates = extract_candidate_keyphrases(processed_text)
    phrase_counts = Counter(candidates)
    ranked_phrases = sorted(phrase_counts.items(), key=lambda x: x[1], reverse=True)
    return [phrase for phrase, _ in ranked_phrases[:num_phrases]]

def split_into_sentences(text):
    return nltk.sent_tokenize(text)

def score_sentences(text, keyphrases, max_words=50):
    sentences = split_into_sentences(text)
    vectorizer = CountVectorizer(vocabulary=keyphrases)
    sentence_vectors = vectorizer.fit_transform(sentences)
    transformer = TfidfTransformer()
    tfidf_matrix = transformer.fit_transform(sentence_vectors)
    sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
    sorted_indices = sentence_scores.argsort()[::-1]

    abstract_sentences = []
    word_count = 0
    for idx in sorted_indices:
        sentence = sentences[idx]
        sentence_word_count = len(sentence.split())
        if word_count + sentence_word_count <= max_words:
            abstract_sentences.append(sentence)
            word_count += sentence_word_count
        else:
            break
    return " ".join(abstract_sentences)

def extract_title_and_abstract():
    # Upload the file
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    # Extract text based on file extension
    extension = file_path.split('.')[-1].lower()
    if extension in ['jpg', 'jpeg', 'png']:
        text = extract_text_from_image(file_path)
    elif extension == 'txt':
        text = extract_text_from_txt(file_path)
    elif extension == 'docx':
        text = extract_text_from_docx(file_path)
    elif extension == 'pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")

    # Generate keyphrases and summary
    keyphrases = rank_keyphrases(text)
    summary = score_sentences(text, keyphrases)

    # Display results as Title and Abstract
    display(HTML(f'''
        <div style="background-color: #f3e5f5; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h2 style="color: #6a1b9a;"><i class="fa fa-book" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Document Title</h2>
            <p style="color: #333; font-size: 18px;">{' '.join(keyphrases[:1])}</p>
        </div>
    '''))

    display(HTML(f'''
        <div style="background-color: #e3f2fd; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h3 style="color: #1976d2;"><i class="fa fa-file-text" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Abstract</h3>
            <p style="color: #555; font-size: 16px;">{summary}</p>
        </div>
    '''))

# Call the function to upload and display Title and Abstract
#extract_title_and_abstract()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Document Similarity

In [8]:


# Define text extraction functions for various file types
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def preprocess_file(file_path):
    extension = os.path.splitext(file_path)[1].lower()
    if extension in ['.jpg', '.jpeg', '.png']:
        return extract_text_from_image(file_path)
    elif extension == '.txt':
        return extract_text_from_txt(file_path)
    elif extension == '.docx':
        return extract_text_from_docx(file_path)
    elif extension == '.pdf':
        return extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")

# Helper functions for keyphrase extraction, ranking, summarization, and similarity
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def extract_candidate_keyphrases(text):
    doc = nlp(text)
    candidates = [chunk.text for chunk in doc.noun_chunks]
    candidates.extend([token.text for token in doc if token.pos_ in {"NOUN", "PROPN", "ADJ"}])
    return candidates

def rank_keyphrases(text, num_phrases=10):
    processed_text = preprocess_text(text)
    candidates = extract_candidate_keyphrases(processed_text)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([processed_text])
    phrase_counts = Counter(candidates)
    ranked_phrases = sorted(phrase_counts.items(), key=lambda x: x[1], reverse=True)
    return [phrase for phrase, _ in ranked_phrases[:num_phrases]]

def split_into_sentences(text):
    return nltk.sent_tokenize(text)

def score_sentences(text, keyphrases, max_words=50):
    sentences = split_into_sentences(text)
    vectorizer = CountVectorizer(vocabulary=keyphrases)
    sentence_vectors = vectorizer.fit_transform(sentences)
    transformer = TfidfTransformer()
    tfidf_matrix = transformer.fit_transform(sentence_vectors)
    sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
    sorted_indices = sentence_scores.argsort()[::-1]
    summary_sentences = []
    word_count = 0
    for idx in sorted_indices:
        sentence = sentences[idx]
        sentence_word_count = len(sentence.split())
        if word_count + sentence_word_count <= max_words:
            summary_sentences.append(sentence)
            word_count += sentence_word_count
        else:
            break
    return " ".join(summary_sentences)

def get_mean_embedding(text, model):
    words = preprocess_text(text).split()
    embeddings = [model.wv[word] for word in words if word in model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

def document_similarity(doc1, doc2, model, lda_model, dictionary, lsa_model, tfidf_vectorizer, keyphrases1, keyphrases2):
    doc1_bow, doc2_bow = dictionary.doc2bow(preprocess_text(doc1).split()), dictionary.doc2bow(preprocess_text(doc2).split())
    doc1_lsa = [topic[1] for topic in lsa_model[doc1_bow]]
    doc2_lsa = [topic[1] for topic in lsa_model[doc2_bow]]
    lsa_similarity = cosine_similarity([doc1_lsa], [doc2_lsa])[0][0]

    doc1_lda = np.array([topic[1] for topic in lda_model.get_document_topics(doc1_bow, minimum_probability=0)])
    doc2_lda = np.array([topic[1] for topic in lda_model.get_document_topics(doc2_bow, minimum_probability=0)])
    lda_similarity = np.dot(doc1_lda, doc2_lda) / (np.linalg.norm(doc1_lda) * np.linalg.norm(doc2_lda))

    keyphrase_set1, keyphrase_set2 = set(keyphrases1), set(keyphrases2)
    jaccard_similarity = len(keyphrase_set1.intersection(keyphrase_set2)) / len(keyphrase_set1.union(keyphrase_set2))

    sentences1, sentences2 = split_into_sentences(doc1), split_into_sentences(doc2)
    tfidf_sentences1, tfidf_sentences2 = tfidf_vectorizer.transform(sentences1), tfidf_vectorizer.transform(sentences2)
    sentence_similarities = cosine_similarity(tfidf_sentences1, tfidf_sentences2)
    sentence_score = np.max(sentence_similarities)

    embedding1, embedding2 = get_mean_embedding(doc1, model), get_mean_embedding(doc2, model)
    embedding_similarity = cosine_similarity([embedding1], [embedding2])[0][0]

    final_score = 0.2 * (lsa_similarity + lda_similarity + jaccard_similarity + sentence_score + embedding_similarity)
    return {
        "LSA Similarity": lsa_similarity,
        "LDA Similarity": lda_similarity,
        "Keyphrase Jaccard Similarity": jaccard_similarity,
        "Sentence-Level Cosine Similarity": sentence_score,
        "Embedding Cosine Similarity": embedding_similarity,
        "Final Document Similarity Score": final_score
    }

def get_html_diff(doc1, doc2):
    doc1, doc2 = re.sub(r'\s+', ' ', doc1).strip(), re.sub(r'\s+', ' ', doc2).strip()
    matcher = SequenceMatcher(None, doc1, doc2)
    html_output = '<div style="font-family: Arial; line-height: 1.5;">'
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'replace':
            html_output += f'<span style="color: red; text-decoration: line-through;">{doc1[i1:i2]}</span>'
            html_output += f'<span style="color: green;">{doc2[j1:j2]}</span>'
        elif tag == 'delete':
            html_output += f'<span style="color: red; text-decoration: line-through;">{doc1[i1:i2]}</span>'
        elif tag == 'insert':
            html_output += f'<span style="color: green;">{doc2[j1:j2]}</span>'
        elif tag == 'equal':
            html_output += f'{doc1[i1:i2]}'
    html_output += '</div>'
    return html_output

# Main similarity comparison function
def similarity_comparison():
    display(HTML('''
    <div style="background-color: #e8f5e9; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #388e3c;">
            <i class="fa fa-balance-scale" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>
            Executing Similarity Comparison...
        </h3>
        <p>Comparing the uploaded documents for similarities using various metrics.</p>
    </div>
    '''))

    print("Please upload the first document:")
    uploaded_files1 = files.upload()
    print("Please upload the second document:")
    uploaded_files2 = files.upload()
    file_path1, file_path2 = next(iter(uploaded_files1)), next(iter(uploaded_files2))

    doc1, doc2 = preprocess_file(file_path1), preprocess_file(file_path2)
    keyphrases1, keyphrases2 = rank_keyphrases(doc1), rank_keyphrases(doc2)
    sentences = [preprocess_text(doc1).split(), preprocess_text(doc2).split()]
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

    dictionary = corpora.Dictionary([preprocess_text(doc1).split(), preprocess_text(doc2).split()])
    corpus = [dictionary.doc2bow(preprocess_text(text).split()) for text in [doc1, doc2]]
    lda_model = models.LdaModel(corpus, num_topics=2, id2word=dictionary)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    lsa_model = models.LsiModel(corpus=corpus, id2word=dictionary, num_topics=2)

    similarity_scores = document_similarity(doc1, doc2, word2vec_model, lda_model, dictionary, lsa_model, tfidf_vectorizer, keyphrases1, keyphrases2)

    similarity_table = '''
    <div style="background-color: #f1f8e9; border-radius: 10px; padding: 20px; margin-top: 20px;">
        <h3 style="color: #33691e;">Similarity Scores</h3>
        <table style="width: 100%; border-collapse: collapse; background-color: #f9fbe7;">
            <tr style="background-color: #8bc34a; color: white; text-align: left;">
                <th style="padding: 10px; font-size: 16px;">Metric</th>
                <th style="padding: 10px; font-size: 16px;">Score</th>
            </tr>'''

    for score_type, score_value in similarity_scores.items():
        row_bg_color = '#dcedc8' if score_value >= 0.3 else '#ffebee'
        text_color = '#33691e' if score_value >= 0.3 else '#d32f2f'
        similarity_table += f'''
            <tr style="border-bottom: 1px solid #ddd; background-color: {row_bg_color};">
                <td style="padding: 10px; font-weight: bold; color: #33691e;">{score_type}</td>
                <td style="padding: 10px; font-weight: bold; color: {text_color};">{score_value:.4f}</td>
            </tr>'''

    similarity_table += '</table></div>'
    display(HTML(similarity_table))

    if similarity_scores['Final Document Similarity Score'] > 0.3:
        html_diff = get_html_diff(doc1, doc2)
        display(HTML('''
        <div style="background-color: #f3e5f5; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h3 style="color: #8e24aa;">Document Differences</h3>
            <p>Highlighted text differences between the two documents:</p>
        </div>'''))
        display(HTML(html_diff))


# Sentiment Analysis

In [9]:
from google.colab import files
from IPython.display import HTML, display
import pytesseract
from PIL import Image
from docx import Document
import fitz  # PyMuPDF for PDFs
from textblob import TextBlob

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        sentiment = "Positive"
    elif polarity < 0:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return sentiment

def perform_sentiment_analysis():
    # Upload the file
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    # Extract text based on file extension
    extension = file_path.split('.')[-1].lower()
    if extension in ['jpg', 'jpeg', 'png']:
        text = extract_text_from_image(file_path)
    elif extension == 'txt':
        text = extract_text_from_txt(file_path)
    elif extension == 'docx':
        text = extract_text_from_docx(file_path)
    elif extension == 'pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")

    # Perform sentiment analysis
    sentiment = analyze_sentiment(text)

    # Display sentiment result
    display(HTML(f'''
        <div style="background-color: #fff3e0; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h3 style="color: #ff9800;"><i class="fa fa-smile-o" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Sentiment Analysis</h3>
            <p style="color: #555; font-size: 16px;">Sentiment: {sentiment}</p>
        </div>
    '''))

# Call the function to upload and display Sentiment Analysis
#perform_sentiment_analysis()


# QnA

In [10]:
# Import necessary libraries
import nltk
import re
import gensim
import numpy as np
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim.models import Word2Vec
import gensim.downloader as api
from docx import Document
import fitz  # PyMuPDF for PDFs
import os
from google.colab import files
from IPython.display import display, HTML, clear_output



# Define functions for document text extraction
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def preprocess_file(file_path):
    extension = os.path.splitext(file_path)[1].lower()
    if extension == '.txt':
        text = extract_text_from_txt(file_path)
    elif extension == '.docx':
        text = extract_text_from_docx(file_path)
    elif extension == '.pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")
    return text

# Define text cleaning functions
def clean_sentence(sentence, stopwords=False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    if stopwords:
        sentence = remove_stopwords(sentence)
    return sentence

def get_cleaned_sentences(tokens, stopwords=False):
    cleaned_sentences = []
    for row in tokens:
        cleaned = clean_sentence(row, stopwords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

# Define functions for word and phrase embeddings
def get_word_vec(word, model):
    try:
        vec = model[word]
    except KeyError:
        vec = np.zeros(model.vector_size)
    return vec

def get_phrase_embedding(phrase, embedding_model):
    samp = get_word_vec('computer', embedding_model)
    vec = np.zeros(len(samp), dtype=float)
    for word in phrase.split():
        vec += get_word_vec(word, embedding_model)
    return vec.reshape(1, -1)

# Function to retrieve and print the FAQ answer
def retrieve_and_print_faq_answer(question_embedding, sent_embeddings, cleaned_sentences):
    similarities = [np.dot(question_embedding, sent_emb.T)[0][0] for sent_emb in sent_embeddings]
    best_match_index = np.argmax(similarities)
    return best_match_index

# Main function to handle file upload and question input in HTML
def process_file_and_answer_question():
    clear_output()
    display(HTML('''
        <div style="background-color: #e8f5e9; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h3 style="color: #388e3c;">Document QA System</h3>
            <p>Please upload a document file (TXT, DOCX, or PDF) and enter your question to find the answer.</p>
        </div>
    '''))

    # Prompt user to upload a file
    print("Please upload a file:")
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    try:
        # Extract text from uploaded file
        text = preprocess_file(file_path)
        print("Extracted Text (Sample):", text[:500], "...")  # Print a sample of the extracted text

        # Tokenize and clean sentences
        tokens = nltk.sent_tokenize(text)
        cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
        sentence_words = [[word for word in document.split()] for document in cleaned_sentences_with_stopwords]

        # Load Word2Vec model from Google Drive
        model_path = '/content/drive/My Drive/w2vecmodel.mod'  # Adjust the path as needed
        if os.path.exists(model_path):
            v2w_model = gensim.models.KeyedVectors.load(model_path)
            print("Word2Vec Model Successfully loaded")
        else:
            v2w_model = api.load('word2vec-google-news-300')
            v2w_model.save(model_path)
            print("Word2Vec Model Saved")


        # Get sentence embeddings
        sent_embeddings = [get_phrase_embedding(sent, v2w_model) for sent in cleaned_sentences_with_stopwords]

        # Prompt user for a question in HTML form
        question_html = '''
            <form action="#" method="post">
                <label for="question">Enter your question:</label><br>
                <input type="text" id="question" name="question" style="width: 100%; padding: 10px; font-size: 16px; margin-top: 10px;"><br><br>
                <input type="button" value="Get Answer" onclick="get_answer()">
            </form>
            <div id="answer_display" style="margin-top: 20px;"></div>
        '''
        display(HTML(question_html))

        # JavaScript to handle question input and display the answer
        display(HTML('''
            <script>
                function get_answer() {
                    const question = document.getElementById('question').value;
                    google.colab.kernel.invokeFunction('answer_callback', [question], {});
                }
            </script>
        '''))

        # Register callback for question input
        def answer_callback(question):
            question = clean_sentence(question, stopwords=False)
            question_embedding = get_phrase_embedding(question, v2w_model)

            # Retrieve answer
            answer_index = retrieve_and_print_faq_answer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)
            answer = cleaned_sentences_with_stopwords[answer_index]

            # Display answer in HTML
            display(HTML(f'<div id="answer_display"><h4 style="color: #388e3c;">Answer:</h4><p style="font-family: Arial; color: #4caf50;">{answer}</p></div>'))

        # Register the callback
        import google.colab
        google.colab.output.register_callback('answer_callback', answer_callback)

    except ValueError as e:
        print(e)

# Execute the main function
#process_file_and_answer_question()


# Paraphrasing

In [11]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import fitz  # PyMuPDF
from docx import Document

# Load models and tokenizers for both Pegasus and T5
def load_models():
    pegasus_model_name = "tuner007/pegasus_paraphrase"
    pegasus_tokenizer = PegasusTokenizer.from_pretrained(pegasus_model_name)
    pegasus_model = PegasusForConditionalGeneration.from_pretrained(pegasus_model_name)

    t5_model_name = 'ramsrigouthamg/t5_paraphraser'
    t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
    t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return pegasus_model, pegasus_tokenizer, t5_model, t5_tokenizer, device

pegasus_model, pegasus_tokenizer, t5_model, t5_tokenizer, device = load_models()

# Function to split text into manageable chunks
def split_text_into_chunks(text, chunk_size=300):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to check if the output length is sufficient
def is_output_long_enough(text, min_length=100):
    return len(text.split()) >= min_length

# Paraphrasing functions
def paraphrase_pegasus(text):
    max_length = pegasus_tokenizer.model_max_length
    inputs = pegasus_tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")

    translated = pegasus_model.generate(
        **inputs,
        max_new_tokens=800,
        num_beams=7,
        early_stopping=False,
        length_penalty=2.0,
        do_sample=False
    )

    paraphrased_text = pegasus_tokenizer.batch_decode(translated, skip_special_tokens=True)
    return paraphrased_text[0]

def paraphrase_t5(text):
    input_text = "paraphrase: " + text
    encoding = t5_tokenizer.encode_plus(input_text, return_tensors="pt", truncation=True, max_length=512)

    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    beam_outputs = t5_model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=800,
        num_beams=7,
        length_penalty=2.0,
        early_stopping=False,
        do_sample=False
    )

    paraphrased_text = t5_tokenizer.decode(beam_outputs[0], skip_special_tokens=True)
    return paraphrased_text

# Function to paraphrase with recursion for longer output
def paraphrase_with_recursion(text, model, min_length=100):
    paraphrased_text = ""
    while not is_output_long_enough(paraphrased_text, min_length):
        paraphrased_text = model(text)
        if len(paraphrased_text) < len(text):  # If output is smaller, stop recursion
            break
        text = paraphrased_text  # Use the paraphrased text for the next iteration
    return paraphrased_text

# Function to paraphrase using Pegasus with chunking
def paraphrase_text_with_pegasus_chunking(text):
    chunks = split_text_into_chunks(text)
    paraphrased_chunks = []

    for chunk in chunks:
        paraphrased_chunk = paraphrase_with_recursion(chunk, paraphrase_pegasus)
        paraphrased_chunks.append(paraphrased_chunk)

    return " ".join(paraphrased_chunks)

# Function to paraphrase using T5 with chunking
def paraphrase_text_with_t5_chunking(text):
    chunks = split_text_into_chunks(text)
    paraphrased_chunks = []

    for chunk in chunks:
        paraphrased_chunk = paraphrase_with_recursion(chunk, paraphrase_t5)
        paraphrased_chunks.append(paraphrased_chunk)

    return " ".join(paraphrased_chunks)

# Text extraction functions
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

# Define the paraphrasing interface function with file upload
def paraphrase_interface():
    model_selection = widgets.Dropdown(
        options=['Select Model', 'Pegasus', 'T5'],
        description='Model:',
        layout=widgets.Layout(width='50%')
    )

    file_upload = widgets.FileUpload(
        accept='.txt, .docx, .pdf',  # Accept text, Word, and PDF files
        multiple=False,
        description="Upload File"
    )

    paraphrase_button = widgets.Button(
        description="Paraphrase",
        layout=widgets.Layout(width='30%'),
        style={'button_color': '#4CAF50'}
    )

    output_display = widgets.Output()

    # Function to handle paraphrasing based on file upload
    def paraphrase_file(b):
        output_display.clear_output()
        with output_display:
            if not file_upload.value:
                print("Error: No file uploaded. Please upload a TXT, DOCX, or PDF file.")
                return

            model_choice = model_selection.value
            if model_choice == 'Select Model':
                print("Error: Please select a paraphrasing model.")
                return

            uploaded_file_name = next(iter(file_upload.value))
            content = file_upload.value[uploaded_file_name]['content']
            file_path = "/tmp/" + uploaded_file_name

            # Save file content locally for processing
            with open(file_path, "wb") as f:
                f.write(content)

            # Extract text based on file extension
            _, ext = os.path.splitext(file_path)
            if ext == '.txt':
                input_text = extract_text_from_txt(file_path)
            elif ext == '.docx':
                input_text = extract_text_from_docx(file_path)
            elif ext == '.pdf':
                input_text = extract_text_from_pdf(file_path)
            else:
                print("Unsupported file type.")
                return

            # Paraphrasing based on selected model
            if model_choice == 'Pegasus':
                paraphrased_text = paraphrase_text_with_pegasus_chunking(input_text)
            elif model_choice == 'T5':
                paraphrased_text = paraphrase_text_with_t5_chunking(input_text)
            else:
                paraphrased_text = "No model selected."

            # Display paraphrased text
            print("Paraphrased Text:")
            print(paraphrased_text)

    # Assign button click event
    paraphrase_button.on_click(paraphrase_file)

    # Display the paraphrasing interface
    display(HTML("""
    <style>
        .widget-label { font-weight: bold; font-size: 16px; }
    </style>
    """))
    display(model_selection, file_upload, paraphrase_button, output_display)

# Call the paraphrasing interface
#paraphrase_interface()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

# Topic Modelling

In [12]:

# Install pytesseract, Tesseract OCR, python-docx, and PyMuPDF
!pip install pytesseract
!apt-get install -y tesseract-ocr
!pip install python-docx
!pip install PyMuPDF

from google.colab import files
from IPython.display import HTML, display
import pytesseract
from PIL import Image
from docx import Document
import fitz  # PyMuPDF for PDFs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def perform_topic_modeling(text, num_topics=3, num_words=5):
    stop_words = stopwords.words('english')
    vectorizer = CountVectorizer(max_df=1.0, min_df=1, stop_words=stop_words)
    text_vectorized = vectorizer.fit_transform([text])

    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(text_vectorized)

    topics = []
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        topic_words = [words[i] for i in topic.argsort()[-num_words:]]
        topics.append(", ".join(topic_words))
    return topics

def perform_topic_modeling_analysis():
    # Upload the file
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    # Extract text based on file extension
    extension = file_path.split('.')[-1].lower()
    if extension in ['jpg', 'jpeg', 'png']:
        text = extract_text_from_image(file_path)
    elif extension == 'txt':
        text = extract_text_from_txt(file_path)
    elif extension == 'docx':
        text = extract_text_from_docx(file_path)
    elif extension == 'pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError(f"Unsupported file format: {extension}")

    # Perform topic modeling
    topics = perform_topic_modeling(text)

    # Display topics in a styled format
    display(HTML(f'''
        <div style="background-color: #e0f7fa; border-radius: 10px; padding: 20px; margin-top: 20px;">
            <h3 style="color: #0097a7;"><i class="fa fa-tags" style="font-size:24px; vertical-align: middle; margin-right: 10px;"></i>Topic Modeling</h3>
            <ul style="color: #555; font-size: 16px;">
                {"".join([f"<li>Topic {i + 1}: {topic}</li>" for i, topic in enumerate(topics)])}
            </ul>
        </div>
    '''))

# Call the function to upload and display Topic Modeling results
perform_topic_modeling_analysis()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,709 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving Cloud_Computing___Final_2023-24_SUsqT65ARv.pdf to Cloud_Computing___Final_2023-24_SUsqT65ARv.pdf


ValueError: empty vocabulary; perhaps the documents only contain stop words

# not needed

In [ ]:
# # Dropdown for selecting functionality
# functionality_dropdown = widgets.Dropdown(
#     options=[
#         ('Select a functionality', ''),
#         ('Knowledge Graph', 'knowledge_graph'),
#         ('Keyphrase Extraction and Summarization', 'keyphrase_extraction_summarization'),
#         ('Similarity Comparison', 'similarity_comparison'),
#         ('Abstract Generation', 'extract_title_and_abstract'),
#         ('Sentiment Analysis', 'perform_sentiment_analysis'),
#         ('Topic Modeling', 'perform_topic_modeling_analysis'),
#         ('Process File and Answer Question', 'process_file_and_answer_question'),
#         ('Paraphrasing', 'paraphrase_interface')

#     ],
#     description='Choose Functionality:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='70%', margin='20px auto')
# )

# # Custom CSS styles
# custom_css = """
# <style>
#     .widget-label {
#         font-size: 16px;
#         font-weight: bold;
#         color: #4a4a4a;
#     }
#     .widget-dropdown {
#         background-color: #f5f5f5;
#         color: #333;
#         font-size: 16px;
#         border: 2px solid #4CAF50;
#         border-radius: 5px;
#         padding: 5px;
#     }
#     .container {
#         max-width: 700px;
#         margin: auto;
#         font-family: Arial, sans-serif;
#     }
#     h2 {
#         color: #1a237e;
#         text-align: center;
#         font-size: 2em;
#     }
#     .instructions {
#         font-size: 1em;
#         color: #555;
#         text-align: center;
#         margin-bottom: 15px;
#     }
# </style>
# """

# # Function to execute selected functionality
# def on_dropdown_change(change):
#     selected_function = change['new']
#     if selected_function:
#         clear_output(wait=True)
#         display(HTML(custom_css))  # Apply custom styles
#         display(HTML('<div class="container">'))
#         display(HTML('<h2>Document Processing System</h2>'))
#         display(HTML('<p class="instructions">Select a functionality from the dropdown to perform the desired task.</p>'))
#         display(functionality_dropdown)
#         display(HTML('</div>'))
#         globals()[selected_function]()


# Execution

In [14]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Dropdown for selecting functionality
functionality_dropdown = widgets.Dropdown(
    options=[
        ('Select a functionality', ''),
        ('Knowledge Graph', 'knowledge_graph'),
        ('Keyphrase Extraction and Summarization', 'keyphrase_extraction_summarization'),
        ('Similarity Comparison', 'similarity_comparison'),
        ('Abstract Generation', 'extract_title_and_abstract'),
        ('Sentiment Analysis', 'perform_sentiment_analysis'),
        ('Topic Modeling', 'perform_topic_modeling_analysis'),
        ('Process File and Answer Question', 'process_file_and_answer_question'),
        ('Paraphrasing', 'paraphrase_interface')
    ],
    description='Choose Functionality:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%', margin='20px auto')
)

# Custom CSS styles
custom_css = """
<style>
    .widget-label {
        font-size: 16px;
        font-weight: bold;
        color: #4a4a4a;
    }
    .widget-dropdown {
        background-color: #f5f5f5;
        color: #333;
        font-size: 16px;
        border: 2px solid #4CAF50;
        border-radius: 5px;
        padding: 5px;
    }
    .container {
        max-width: 700px;
        margin: auto;
        font-family: Arial, sans-serif;
    }
    h2 {
        color: #1a237e;
        text-align: center;
        font-size: 2em;
    }
    .instructions {
        font-size: 1em;
        color: #555;
        text-align: center;
        margin-bottom: 15px;
    }
</style>
"""

# Function to execute selected functionality
def on_dropdown_change(change):
    selected_function = change['new']
    if selected_function:
        clear_output(wait=True)
        display(HTML(custom_css))  # Apply custom styles
        display(HTML('<div class="container">'))
        display(HTML('<h2>Document Processing System</h2>'))
        display(HTML('<p class="instructions">Select a functionality from the dropdown to perform the desired task.</p>'))
        display(functionality_dropdown)
        display(HTML('</div>'))

        # Call the appropriate function based on the selected option
        if selected_function == 'knowledge_graph':
            knowledge_graph()  # Replace with your actual function
        elif selected_function == 'keyphrase_extraction_summarization':
            keyphrase_extraction_summarization()  # Replace with your actual function
        elif selected_function == 'similarity_comparison':
            similarity_comparison()  # Replace with your actual function
        elif selected_function == 'extract_title_and_abstract':
            extract_title_and_abstract()  # Replace with your actual function
        elif selected_function == 'perform_sentiment_analysis':
            perform_sentiment_analysis()  # Replace with your actual function
        elif selected_function == 'perform_topic_modeling_analysis':
            perform_topic_modeling_analysis()  # Your function
        elif selected_function == 'process_file_and_answer_question':
            process_file_and_answer_question()  # Your function
        elif selected_function == 'paraphrase_interface':
            paraphrase_interface()  # Replace with your actual function

# Trigger function execution when dropdown value changes
functionality_dropdown.observe(on_dropdown_change, names='value')

# Display the dropdown with custom HTML styling
display(HTML(custom_css))  # Display custom CSS styles
display(HTML('<div class="container">'))
display(HTML('<h2>Document Processing System</h2>'))
display(HTML('<p class="instructions">Choose a functionality to execute below:</p>'))
display(functionality_dropdown)
display(HTML('</div>'))


Dropdown(description='Choose Functionality:', index=5, layout=Layout(margin='20px auto', width='70%'), options…

Saving How to use Loss function.docx to How to use Loss function.docx
